In [10]:
import pandas as pd
import sqlite3
import os
import glob
import nltk

path = "/Users/brentpeiffer/Library/Messages/chat.db"
conn = sqlite3.connect(path)

df = pd.read_sql_query('SELECT datetime (message.date / 1000000000 + strftime ("%s", "2001-01-01"), "unixepoch", "localtime") AS message_date, message.text, message.is_from_me, message.associated_message_guid, chat.chat_identifier, handle.id as handle_id FROM chat JOIN chat_message_join ON chat. "ROWID" = chat_message_join.chat_id JOIN message ON chat_message_join.message_id = message. "ROWID" LEFT JOIN handle on message.handle_id = handle. "ROWID" WHERE message.text not null ORDER BY message_date ASC', conn)
df.tail()




,message_date,text,is_from_me,associated_message_guid,chat_identifier,handle_id
27547,2020-12-05 17:24:51,No package yet. They usually deliver up until ...,0,None,chat21811412373658541,+13154157338
27548,2020-12-05 18:03:58,hiiiii buying anything nice??,1,None,+12032585295,+12032585295
27549,2020-12-05 18:45:30,What does this reply look like,1,None,+12032585295,+12032585295
27550,2020-12-05 19:03:31,is_emote?,1,None,+12032585295,+12032585295
27551,2020-12-05 19:04:39,Whattttt,0,None,+12032585295,+12032585295


In [13]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
scores = df['text'].apply(vader.polarity_scores).tolist()
scores[:5]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/brentpeiffer/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


[{'neg': 0.0, 'neu': 0.265, 'pos': 0.735, 'compound': 0.8519},
 {'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]

In [16]:
scores_df = pd.DataFrame(scores)
# join it with main dataframe
df = df.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
df['message_date'] = pd.to_datetime(df.message_date).dt.date
# Group by date and ticker columns from scored_news and calculate the mean
mean_scores = df.groupby(['message_date']).mean()
mean_scores.tail()

,is_from_me,neg,neu,pos,compound,neg_right,neu_right,pos_right,compound_right,neg_right,neu_right,pos_right,compound_right
message_date,,,,,,,,,,,,,
2020-12-01,0.331897,0.006728,0.433250,0.094509,0.094572,0.006728,0.433250,0.094509,0.094572,0.006728,0.433250,0.094509,0.094572
2020-12-02,0.281250,0.046510,0.693021,0.224010,0.215807,0.046510,0.693021,0.224010,0.215807,0.046510,0.693021,0.224010,0.215807
2020-12-03,0.404762,0.048643,0.618643,0.285048,0.307190,0.048643,0.618643,0.285048,0.307190,0.048643,0.618643,0.285048,0.307190
2020-12-04,0.289474,0.050632,0.674816,0.248263,0.321179,0.050632,0.674816,0.248263,0.321179,0.050632,0.674816,0.248263,0.321179
2020-12-05,0.259259,0.043667,0.710593,0.208667,0.137781,0.043667,0.710593,0.208667,0.137781,0.043667,0.710593,0.208667,0.137781
